# Project Description

The goal is to create a web application, introducing a trading algorithm which gives a suggestion to buy, not buy, sell, or hold, and then executes that trade through user input.

Trading platform will be Alpaca. Server will be on Replit. Web App will be on Anvil. See links below.

The algorithm will look at historical stock data 1 year prior to current date. It will use the parameters which can be retrieved from Alpaca to train the model. 

The model will also include sentiment score as an independent variable, which means historical articles from each day will need to be pulled to give an average sentiment score. 

The model will be trained on the top 10 stocks in 2024 in the S&P 500 by index weight. 

The model will output a prediction for the next week. Based on its prediction, the model will suggest to buy, not buy, sell, or hold.

# Links

https://anvil.works/

https://app.alpaca.markets/paper/dashboard/overview

https://replit.com/

# Login Information

Email: nshuckerow@ucsd.edu

Password: MGTA415data!

In [57]:
import os
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta
import pandas as pd

# Alpaca API credentials
ALPACA_API_KEY = "PKDFQIPQCXE38TAG1WKG"
ALPACA_SECRET_KEY = "LRaWwamsMm0WDa58x8S0z8wje9gRcYhcgtMf7C55"
ALPACA_URL = 'https://paper-api.alpaca.markets'

# Initialize Alpaca API
alpaca = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, base_url=ALPACA_URL, api_version='v2')




                         time     open     high      low    close  volume
0   2023-08-21 00:00:00-04:00  317.855  322.725  317.130  321.940  408350
1   2023-08-22 00:00:00-04:00  325.540  325.900  321.520  322.450  326147
2   2023-08-23 00:00:00-04:00  323.770  329.190  323.480  326.975  311403
3   2023-08-24 00:00:00-04:00  332.870  332.870  319.990  320.040  473996
4   2023-08-25 00:00:00-04:00  321.595  325.075  318.900  322.860  303301
..                        ...      ...      ...      ...      ...     ...
246 2024-08-13 00:00:00-04:00  409.610  414.880  409.610  413.780  247042
247 2024-08-14 00:00:00-04:00  415.190  417.690  412.565  416.655  231308
248 2024-08-15 00:00:00-04:00  419.610  421.085  417.690  420.870  320490
249 2024-08-16 00:00:00-04:00  420.775  420.945  417.330  418.500  192582
250 2024-08-19 00:00:00-04:00  419.080  421.560  416.480  421.450  211362

[251 rows x 6 columns]


In [80]:
def get_historical_stock_data(symbols, start_date, end_date):
    # Use the correct TimeFrame object for daily data
    timeframe = tradeapi.TimeFrame.Day
    
    all_data = []
    
    for symbol in symbols:
        bars = alpaca.get_bars(
            symbol,
            timeframe=timeframe,
            start=start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
            end=end_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
            adjustment='raw',
            feed='iex'
        )
        
        data = []
        for bar in bars:
            data.append({
                'symbol': symbol,  # Add the symbol to the data
                'time': bar.t,
                'open': bar.o,
                'high': bar.h,
                'low': bar.l,
                'close': bar.c,
                'volume': bar.v
            })
        
        all_data.extend(data)
    
    return pd.DataFrame(all_data)

In [81]:
top_50_sp500_stocks = [
    'AAPL',  # Apple Inc.
    'MSFT',  # Microsoft Corporation
    'AMZN',  # Amazon.com Inc.
    'NVDA',  # NVIDIA Corporation
    'GOOGL', # Alphabet Inc. (Class A)
    'GOOG',  # Alphabet Inc. (Class C)
    'TSLA',  # Tesla Inc.
    'META',  # Meta Platforms Inc.
    'BRK.B', # Berkshire Hathaway Inc. (Class B)
    'UNH',   # UnitedHealth Group Incorporated
    'JNJ',   # Johnson & Johnson
    'XOM',   # Exxon Mobil Corporation
    'V',     # Visa Inc.
    'PG',    # Procter & Gamble Co.
    'JPM',   # JPMorgan Chase & Co.
    'LLY',   # Eli Lilly and Company
    'MA',    # Mastercard Incorporated
    'HD',    # The Home Depot Inc.
    'CVX',   # Chevron Corporation
    'MRK',   # Merck & Co. Inc.
    'PEP',   # PepsiCo Inc.
    'ABBV',  # AbbVie Inc.
    'KO',    # The Coca-Cola Company
    'PFE',   # Pfizer Inc.
    'AVGO',  # Broadcom Inc.
    'COST',  # Costco Wholesale Corporation
    'MCD',   # McDonald's Corporation
    'TMO',   # Thermo Fisher Scientific Inc.
    'WMT',   # Walmart Inc.
    'DHR',   # Danaher Corporation
    'NKE',   # NIKE Inc.
    'DIS',   # The Walt Disney Company
    'ADBE',  # Adobe Inc.
    'NFLX',  # Netflix Inc.
    'VZ',    # Verizon Communications Inc.
    'CSCO',  # Cisco Systems Inc.
    'ABT',   # Abbott Laboratories
    'ACN',   # Accenture plc
    'NEE',   # NextEra Energy Inc.
    'LIN',   # Linde plc
    'TXN',   # Texas Instruments Incorporated
    'MDT',   # Medtronic plc
    'PM',    # Philip Morris International Inc.
    'WFC',   # Wells Fargo & Company
    'HON',   # Honeywell International Inc.
    'QCOM',  # QUALCOMM Incorporated
    'BMY',   # Bristol-Myers Squibb Company
    'LOW',   # Lowe's Companies Inc.
    'UNP',   # Union Pacific Corporation
    'RTX'    # Raytheon Technologies Corporation
]


In [93]:
# Retrieve historical stock data for the past year
end_date = datetime.utcnow()
start_date = end_date - timedelta(days=30)
historical_data = get_historical_stock_data(top_50_sp500_stocks, start_date, end_date)

print(historical_data)

     symbol                      time     open     high      low    close  \
0      AAPL 2024-07-22 00:00:00-04:00  227.335  227.770  223.180  224.160   
1      AAPL 2024-07-23 00:00:00-04:00  224.365  226.925  222.680  224.920   
2      AAPL 2024-07-24 00:00:00-04:00  224.115  224.765  217.165  218.585   
3      AAPL 2024-07-25 00:00:00-04:00  218.880  220.810  214.640  217.420   
4      AAPL 2024-07-26 00:00:00-04:00  218.940  219.480  216.040  218.030   
...     ...                       ...      ...      ...      ...      ...   
1045    RTX 2024-08-13 00:00:00-04:00  116.890  117.165  116.070  116.830   
1046    RTX 2024-08-14 00:00:00-04:00  116.760  117.800  116.695  117.640   
1047    RTX 2024-08-15 00:00:00-04:00  118.430  118.775  117.535  118.360   
1048    RTX 2024-08-16 00:00:00-04:00  117.930  118.410  117.575  117.960   
1049    RTX 2024-08-19 00:00:00-04:00  117.975  118.520  117.975  118.350   

       volume  
0      472525  
1      465219  
2     1005989  
3      6427

# Use Alpaca API to return News Articles

The get_news function returns articles for the specified period. The function takes in a stock symbol, the max return results, and number of days. The default number of days is 1, and default max results is 200.

In [103]:
import requests
from datetime import datetime, timedelta
import time

def get_news(symbols, days_prior=1):
    today = datetime.utcnow()
    yesterday = today - timedelta(days=days_prior)  # Adjusted to retrieve news from the previous day
    url = "https://data.alpaca.markets/v1beta1/news"
    headers = {
        "APCA-API-KEY-ID": ALPACA_API_KEY,
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY
    }
    news = []
    page_token = None
    
    while True:
        params = {
            "symbols": ",".join(symbols),
            "start": start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
            "end": today.strftime('%Y-%m-%dT%H:%M:%SZ'),
            "limit": 50,  # Maximum allowed by the API
            "page_token": page_token  # For pagination
        }
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            result = response.json()
            
            news.extend(result.get('news', []))
            
            # Get the next page token if available
            page_token = result.get('next_page_token')
            
            if not page_token:  # No more pages to fetch
                break
            
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                print("Rate limit reached. Sleeping for 10 seconds...")
                time.sleep(1)  # Wait for 60 seconds before retrying
            else:
                print(f"HTTP error occurred: {e}")
                break
        except Exception as e:
            print(f"An error occurred: {e}")
            break
        
    return news

# Example usage

# Retrieve news for specific symbols

news = get_news(top_50_sp500_stocks, days_prior=30)



In [104]:
news_df = pd.DataFrame(news)

# list all unique dates (not times) in news_df

unique_dates = news_df['created_at'].apply(lambda x: x.split('T')[0]).unique()

print(unique_dates)

['2024-08-19' '2024-08-18' '2024-08-17' '2024-08-16' '2024-08-05'
 '2024-08-15' '2024-08-12' '2024-08-14' '2024-08-09' '2024-08-13'
 '2024-08-03' '2024-08-11' '2024-08-10' '2024-07-22' '2024-08-08'
 '2024-08-07' '2024-08-06' '2024-08-02' '2024-08-04' '2024-08-01'
 '2024-07-31' '2024-07-30' '2024-06-18' '2024-07-29' '2023-12-01'
 '2024-07-28' '2024-07-27' '2024-04-29' '2024-07-26' '2024-07-24'
 '2024-07-25' '2024-07-23' '2024-07-21']


In [106]:
from transformers import pipeline, BertTokenizer, BertForSequenceClassification
import pandas as pd

# Load the model and tokenizer
model_name = "yiyanghkust/finbert-tone"  # Example of a financial sentiment model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [107]:
# Function to split text into 512-token chunks based on tokenization
def split_into_chunks(text, max_length=500):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)['input_ids'][0]
    # Ensure each chunk is no more than 512 tokens
    chunks = []
    for i in range(0, len(tokens), max_length):
        chunk = tokens[i:i + max_length]
        # Make sure the chunk is exactly 512 tokens or less
        if len(chunk) > max_length:
            chunk = chunk[:max_length]
        chunks.append(chunk)
    return chunks

In [108]:
# Function to analyze sentiment for long texts
def analyze_sentiment(text):
    chunks = split_into_chunks(text)
    sentiments = []
    for chunk in chunks:
        # Convert tokens back to text before sentiment analysis
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        sentiments.append(nlp(chunk_text)[0])
    
    # Aggregate sentiment scores (e.g., by averaging)
    avg_sentiment_score = sum(s['score'] for s in sentiments) / len(sentiments)
    # Determine overall sentiment by majority vote or averaging
    positive_scores = sum(s['score'] for s in sentiments if s['label'] == 'positive')
    negative_scores = sum(s['score'] for s in sentiments if s['label'] == 'negative')
    sentiment_label = 'positive' if positive_scores >= negative_scores else 'negative'
    
    return sentiment_label, avg_sentiment_score

In [63]:
# Function to calculate the overall sentiment score
def calculate_overall_sentiment(row):
    content_label, content_score = analyze_sentiment(row['content'])
    headline_label, headline_score = analyze_sentiment(row['headline'])
    
    # Average the sentiment scores of content and headline
    overall_score = (content_score + headline_score) / 2
    overall_label = 'positive' if (content_score + headline_score) >= 1 else 'negative'
    
    return overall_label, overall_score

In [109]:
# Apply the overall sentiment analysis function to your DataFrame
news_df['sentiment'], news_df['sentiment_score'] = zip(*news_df['headline'].apply(analyze_sentiment))

In [110]:
import pandas as pd

# Assume your DataFrame is named `news_df`

# Step 1: Convert 'created_at' column to datetime format
news_df['created_at'] = pd.to_datetime(news_df['created_at'])

# Step 2: Extract the date (without the time) from the 'created_at' column
news_df['date'] = news_df['created_at'].dt.date

# Step 3: Group by the extracted date and calculate the average sentiment score
daily_sentiment = news_df.groupby('date')['sentiment_score'].mean().reset_index()

# Step 4: Rename the columns for clarity
daily_sentiment.columns = ['date', 'average_sentiment_score']

# Display the result
print(daily_sentiment)


          date  average_sentiment_score
0   2023-12-01                 0.999948
1   2024-04-29                 0.961141
2   2024-06-18                 0.510231
3   2024-07-21                 0.962439
4   2024-07-22                 0.969042
5   2024-07-23                 0.960296
6   2024-07-24                 0.952226
7   2024-07-25                 0.961484
8   2024-07-26                 0.963447
9   2024-07-27                 0.837638
10  2024-07-28                 0.999872
11  2024-07-29                 0.949442
12  2024-07-30                 0.963917
13  2024-07-31                 0.954561
14  2024-08-01                 0.956118
15  2024-08-02                 0.954622
16  2024-08-03                 0.952031
17  2024-08-04                 0.909672
18  2024-08-05                 0.955715
19  2024-08-06                 0.967089
20  2024-08-07                 0.978305
21  2024-08-08                 0.939025
22  2024-08-09                 0.978020
23  2024-08-10                 0.916422


In [69]:
import pandas as pd

# Assuming `historical_data` is your DataFrame containing historical stock data
# Convert the 'time' column to datetime if it's not already
historical_data['time'] = pd.to_datetime(historical_data['time'])

# Extract the date part from the 'time' column
historical_data['date'] = historical_data['time'].dt.date

# Calculate the daily percentage change in closing price
historical_data['price_change_pct'] = historical_data['close'].pct_change() * 100

# Remove the first row since it will have NaN for the percentage change
historical_data = historical_data.dropna(subset=['price_change_pct'])

# Select only relevant columns
historical_data = historical_data[['date', 'price_change_pct']]


In [70]:
historical_data

,date,price_change_pct
1,2023-08-22,0.158415
2,2023-08-23,1.403318
3,2023-08-24,-2.120957
4,2023-08-25,0.881140
5,2023-08-28,0.312829
...,...,...
246,2024-08-13,1.678339
247,2024-08-14,0.694814
248,2024-08-15,1.011628
249,2024-08-16,-0.563119


In [71]:
# Assuming `daily_sentiment` is your DataFrame containing average sentiment scores by date

# Merge the sentiment data with the stock price data on the 'date' column
merged_df = pd.merge(daily_sentiment, historical_data, on='date', how='inner')

# Display the merged DataFrame
print(merged_df)


          date  average_sentiment_score  price_change_pct
0   2024-07-30                 0.969766         -0.944435
1   2024-07-31                 0.995069         -1.102489
2   2024-08-01                 0.891831         -0.236831
3   2024-08-02                 0.991313         -2.040621
4   2024-08-05                 0.938245         -3.214041
5   2024-08-06                 0.943883          1.019247
6   2024-08-07                 0.985208         -0.207801
7   2024-08-08                 0.935992          1.111418
8   2024-08-09                 0.974452          0.741899
9   2024-08-12                 0.949040          0.231521
10  2024-08-13                 0.985811          1.678339
11  2024-08-14                 0.974394          0.694814
12  2024-08-15                 0.974634          1.011628
13  2024-08-16                 0.961011         -0.563119
14  2024-08-19                 0.976414          0.704898


In [52]:
# normalize the average sentiment score

from sklearn.preprocessing import MinMaxScaler

# Normalize the 'average_sentiment_score' and 'price_change_pct' columns

scaler = MinMaxScaler()
merged_df[['normalized_sentiment', 'normalized_price_change']] = scaler.fit_transform(merged_df[['average_sentiment_score', 'price_change_pct']])
print(merged_df)

          date  average_sentiment_score  price_change_pct  \
0   2024-07-30                 0.904350         -0.944435   
1   2024-07-31                 0.925414         -1.102489   
2   2024-08-01                 0.962532         -0.236831   
3   2024-08-02                 0.868525         -2.040621   
4   2024-08-05                 0.981000         -3.214041   
5   2024-08-06                 0.941374          1.019247   
6   2024-08-07                 0.965586         -0.207801   
7   2024-08-08                 0.928285          1.111418   
8   2024-08-09                 0.933163          0.741899   
9   2024-08-12                 0.897671          0.231521   
10  2024-08-13                 0.938136          1.678339   
11  2024-08-14                 0.940602          0.694814   
12  2024-08-15                 0.919611          1.011628   
13  2024-08-16                 0.947846         -0.563119   
14  2024-08-19                 0.997491          0.704898   

    normalized_sentimen

In [78]:
# Calculate the correlation between sentiment score and stock price change
correlation = merged_df['average_sentiment_score'].corr(merged_df['price_change_pct'])

print(f'Correlation between sentiment score and stock price change: {correlation}')


Correlation between sentiment score and stock price change: 0.03302646570894096
